In [7]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

--2024-03-15 17:54:51--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  7.44MB/s    in 3.5s    

2024-03-15 17:54:55 (7.10 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
books_filename = 'KNN/BX-Books.csv'
ratings_filename = 'KNN/BX-Book-Ratings.csv'

In [45]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [53]:
df_ratings.dropna(inplace=True)

user_rating_count = df_ratings["user"].value_counts()
book_rating_count = df_ratings["isbn"].value_counts()

user_with_more_than_200_ratings = user_rating_count[user_rating_count >= 200].index
books_with_more_than_100_ratings = book_rating_count[book_rating_count >= 100].index

df_filtered_ratings = df_ratings[(df_ratings["user"].isin(user_with_more_than_200_ratings)) & (df_ratings["isbn"].isin(books_with_more_than_100_ratings))]
df_filtered_ratings = df_books.merge(df_filtered_ratings, on="isbn")

In [54]:
df_filtered_ratings.head()

,isbn,title,author,user,rating
0,0440234743,The Testament,John Grisham,277478,0.0
1,0440234743,The Testament,John Grisham,2977,0.0
2,0440234743,The Testament,John Grisham,3363,0.0
3,0440234743,The Testament,John Grisham,7346,9.0
4,0440234743,The Testament,John Grisham,9856,0.0


In [56]:
df = pd.pivot_table(df_filtered_ratings, index="title", columns="user", values="rating", fill_value=0, sort=True)

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
knn = NearestNeighbors(metric='cosine',n_neighbors=5, algorithm='brute')
knn.fit(df)

NearestNeighbors(algorithm='brute', metric='cosine')

In [107]:
def get_recommends(title):
    try:
        book = df.loc[title]
    except KeyError as e:
        print("The given book:", title, "does not exist in dataset.")

    distance, indecies = knn.kneighbors([book.values], n_neighbors=6)
    recommended_books = pd.DataFrame({
        'title': df.iloc[indecies[0]].index.values,
        'distance': distance[0]
    }).sort_values(by='distance', ascending=False).head(5).values

    return [title, recommended_books]

In [109]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("KNN passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("Test failed.")

test_book_recommendation()

KNN passed the challenge! 🎉🎉🎉🎉🎉


In [110]:
# Use tested model
book_to_predict = input()

print(get_recommends(book_to_predict))

 The Surgeon


['The Surgeon', array([["Where the Heart Is (Oprah's Book Club (Paperback))",
        0.7699410915374756],
       ['Seinlanguage', 0.7671219110488892],
       ['Manhattan Hunt Club', 0.7659204602241516],
       ['Last Man Standing', 0.7655158638954163],
       ['The Honk and Holler Opening Soon', 0.7648533582687378]],
      dtype=object)]
